# Buffer1

In [1]:
import pandas as pd

In [2]:
merged_data=pd.read_csv('../data/processed/well_pchip.csv')
gage_to_wells_df = pd.read_csv('../data/processed/wells_with_catchment_info.csv')

In [3]:
import pandas as pd

# Assuming gage_to_wells_df is already loaded
# Convert all column names to lowercase
gage_to_wells_df.columns = gage_to_wells_df.columns.str.lower()

# Drop the specified columns
gage_to_wells_df.drop(columns=['name', 'river', 'state', 'aquifer_na'], inplace=True, errors='ignore')

# Rename the specified columns
gage_to_wells_df.rename(columns={
    'latitude': 'gage_lat',
    'longitude': 'gage_lon',
    'lat_dec': 'well_lat',
    'long_dec': 'well_lon'
}, inplace=True)

# Display the updated DataFrame
print(gage_to_wells_df.head())


        well_id          well_name   well_lat    well_lon     gse  \
0  3.946181e+14  (D-12- 4)16ccb- 1  39.771627 -111.488248  5980.0   
1  3.946341e+14  (D-12- 4)16bcc- 1  39.776071 -111.488248  5960.0   
2  3.946431e+14  (D-12- 4)16bcb- 1  39.778571 -111.487970  5955.0   
3  3.946491e+14  (D-12- 4)17abd- 1  39.780238 -111.494915  5938.0   
4  3.947461e+14  (D-12- 4) 9bab- 1  39.796071 -111.482970  5977.0   

                           geometry  catchment_id     gage_id  \
0   POINT (-111.4882484 39.7716267)   710648999.0  10152000.0   
1   POINT (-111.4882483 39.7760711)   710647014.0  10152000.0   
2  POINT (-111.4879704 39.77857106)   710647014.0  10152000.0   
3  POINT (-111.4949154 39.78023766)   710549798.0  10152000.0   
4  POINT (-111.4829697 39.79607075)   710647014.0  10152000.0   

                             gage_name  
0  SPANISH FORK NEAR LAKE SHORE - UTAH  
1  SPANISH FORK NEAR LAKE SHORE - UTAH  
2  SPANISH FORK NEAR LAKE SHORE - UTAH  
3  SPANISH FORK NEAR LAKE SHOR

In [4]:
well_pchip=merged_data.copy()

In [6]:
import pandas as pd

# Assuming well_pchip and gage_to_wells_df are already loaded
# Perform an inner merge to filter and combine the data based on 'well_id'
filtered_data = pd.merge(
    well_pchip,
    gage_to_wells_df[['gage_id', 'well_id', 'well_lat', 'well_lon']],
    on='well_id',
    how='inner'
)

# Drop the duplicate columns with suffixes
filtered_data.drop(columns=['well_lat_x', 'well_lon_x'], inplace=True)

# Optionally, rename the remaining columns if needed
filtered_data.rename(columns={'well_lat_y': 'well_lat', 'well_lon_y': 'well_lon'}, inplace=True)

# Display the new DataFrame
print(filtered_data.head())


           well_id        date          wte     gse     gage_id   well_lat  \
0  394643111291401  1971-09-15  5953.300000  5955.0  10152000.0  39.778571   
1  394643111291401  1971-09-16  5953.312390  5955.0  10152000.0  39.778571   
2  394643111291401  1971-09-17  5953.324794  5955.0  10152000.0  39.778571   
3  394643111291401  1971-09-18  5953.337211  5955.0  10152000.0  39.778571   
4  394643111291401  1971-09-19  5953.349641  5955.0  10152000.0  39.778571   

    well_lon  
0 -111.48797  
1 -111.48797  
2 -111.48797  
3 -111.48797  
4 -111.48797  


In [7]:
gage_info = pd.read_csv('../data/raw/streamflow/gsl_nwm_gage.csv')
gage_info

,id,name,River,latitude,longitude,elevation_m,state
0,10011200,WEST FORK BEAR RIVER AT WHITNEY DAM - NR OAKLE...,WEST FORK BEAR RIVER,40.841614,-110.927119,2797,Utah
1,10011500,BEAR RIVER NEAR UTAH-WYOMING STATE LINE,BEAR RIVER,40.965225,-110.853508,2432,Utah
2,10015700,SULPHUR CR.AB.RES.BL.LA CHAPELLE CR.NR EVANSTO...,SULPHUR CREEK,41.129114,-110.806563,2205,Wyoming
3,10015900,SULPHUR CREEK BL RES. - NEAR EVANSTON - WYO.,SULPHUR CREEK,41.156058,-110.835176,2173,Wyoming
4,10016900,BEAR RIVER AT EVANSTON - WY,BEAR RIVER,41.270224,-110.963793,2057,Wyoming
...,...,...,...,...,...,...,...
73,10168500,BIG COTTONWOOD CR NR SALT LAKE CITY UTAH,LITTLE COTTONWOOD CREEK,40.618559,-111.781876,1527,Utah
74,10171000,JORDAN RIVER @ 1700 SOUTH @ SALT LAKE CITY - UT,JORDAN RIVER,40.733557,-111.923270,1294,Utah
75,10172700,VERNON CREEK NEAR VERNON - UT,VERNON CREEK,39.979391,-112.380230,1898,Utah
76,10172860,WARM CREEK NEAR GANDY - UT,WARM CREEK,39.459528,-114.023722,1573,Utah


In [9]:
# Merge gage_info with filtered_data to add gage coordinates
filtered_data = pd.merge(
    filtered_data,
    gage_info[['id', 'latitude', 'longitude']],
    left_on='gage_id',
    right_on='id',
    how='left'
)

# Rename the columns
filtered_data.rename(columns={
    'latitude': 'gage_lat',
    'longitude': 'gage_lon'
}, inplace=True)

# Drop the redundant id column
filtered_data.drop('id', axis=1, inplace=True)


In [10]:
filtered_data.to_csv('../data/processed/after_buffer1.csv', index=False)

In [12]:
# Calculate the number of unique wells
unique_wells_count = filtered_data['well_id'].nunique()

# Display the result
print(f"Number of unique wells: {unique_wells_count}")

unique_gages_count = filtered_data['gage_id'].nunique()
print(f"Number of unique gages: {unique_gages_count}")

Number of unique wells: 635
Number of unique gages: 7


# buffer2

In [21]:
import pandas as pd
import numpy as np

def filter_and_analyze_wte(filtered_data, merged_df, distance_buffer_meters=30, delta_bins=None):
    # Convert WTE from feet to meters
    filtered_data['wte_meters'] = filtered_data['wte'] * 0.3048

    # Merge filtered_data with merged_df to get reach_elev
    merged_data = pd.merge(
        filtered_data,
        merged_df[['well_id', 'reach_elev_m']],
        on='well_id',
        how='inner'
    )

    # Calculate the difference between reach elevation and WTE
    merged_data['delta_elev'] = merged_data['reach_elev_m'] - merged_data['wte_meters']

    # Filter out values where the difference is greater than the distance buffer
    filtered_result = merged_data[merged_data['delta_elev'] <= distance_buffer_meters]

    # Define default delta bins if not provided
    if delta_bins is None:
        delta_bins = [-float('inf'), -20, -10, -5, 0, 5, 10, 20, 30, 50, 75, 100, float('inf')]

    # Bin the delta_elev values
    bin_labels = [f"< {delta_bins[1]}"] + \
                 [f"{delta_bins[i]} to {delta_bins[i+1]}" for i in range(1, len(delta_bins)-2)] + \
                 [f">= {delta_bins[-2]}"]
    filtered_result['delta_bin'] = pd.cut(filtered_result['delta_elev'], bins=delta_bins, labels=bin_labels)

    # Calculate statistics
    total_measurements = len(filtered_result)
    dist_stats = filtered_result.groupby('delta_bin').size().reset_index(name='count')
    dist_stats['percentage'] = (dist_stats['count'] / total_measurements * 100).round(2)

    # Display statistics
    print(dist_stats)

    return filtered_result, dist_stats

# Example usage
filtered_data = pd.read_csv('../data/processed/after_buffer1.csv')  # Load your filtered data
merged_df = pd.read_csv('../data/processed/well_reach.csv')  # Load the merged data with reach_elev
# filtered_result, dist_stats = filter_and_analyze_wte(filtered_data, merged_df, distance_buffer_meters=30)


In [22]:
filtered_result, dist_stats = filter_and_analyze_wte(filtered_data, merged_df, distance_buffer_meters=10000)

     delta_bin  count  percentage
0        < -20      0         NaN
1   -20 to -10      0         NaN
2    -10 to -5      0         NaN
3      -5 to 0      0         NaN
4       0 to 5      0         NaN
5      5 to 10      0         NaN
6     10 to 20      0         NaN
7     20 to 30      0         NaN
8     30 to 50      0         NaN
9     50 to 75      0         NaN
10   75 to 100      0         NaN
11      >= 100      0         NaN


/var/folders/fk/mzjttwrn7rn5nvcfhpgm4mrc0000gq/T/ipykernel_13546/3223962800.py:34: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dist_stats = filtered_result.groupby('delta_bin').size().reset_index(name='count')


In [18]:
filtered_result

,well_id,date,wte,gse,gage_id,well_lat,well_lon,gage_lat,gage_lon,wte_meters,reach_elev_m,delta_elev,delta_bin
